This code is intended to be a simple walk-through of a model output and draws on some default plotting functions

In [ ]:
from pygem_eb.processing.plotting_fxns import *
import xarray as xr

Open the file and do a quick inspection of the data variables contained within

In [ ]:
output_fn = '../Output/YOUR_FILE_NAME_HERE.nc'
ds = xr.open_dataset(output_fn)
ds

Visualize the snow layers for the duration of the model simulation

In [ ]:
# Define the variables to plot 
# Options: layertemp, layerdensity, layerwater, layergrainsize layerBC, layerOC, layerdust
layer_vars = ['layertemp','layerdensity','layerwater']

# Find the start and end timestamp
start = ds.time.values[0]
end = ds.time.values[-1]

# Create a daily timeseries to sample the model output
daily_dates = pd.date_range(start, end)

# Plot the layers
visualize_layers(ds, daily_dates, layer_vars)